In [ ]:
import os
import shutil

# 定义两个数据集的源路径
datasets_config = {
    "Self": {
        "img": r'H:\TCV_PROJECT\dataset\self capture img\images\train',
        "lbl": r'H:\TCV_PROJECT\dataset\self capture img\labels\train',
        "out": r'H:\TCV_PROJECT\dataset\self capture img\resnet_train_format'
    },
    "Public": {
        "img": r'H:\TCV_PROJECT\dataset\kagel img\images\train',
        "lbl": r'H:\TCV_PROJECT\dataset\kagel img\labels\train',
        "out": r'H:\TCV_PROJECT\dataset\kagel img\resnet_train_format'
    }
}

class_names = ['glass', 'metal', 'paper', 'plastic']

for ds_name, paths in datasets_config.items():
    for cls in class_names:
        os.makedirs(os.path.join(paths["out"], cls), exist_ok=True)
    
    for label_file in os.listdir(paths["lbl"]):
        if not label_file.endswith('.txt'): continue
        with open(os.path.join(paths["lbl"], label_file), 'r') as f:
            line = f.readline()
            if not line: continue
            class_id = int(line.split()[0])
        
        img_name = label_file.replace('.txt', '.jpg')
        src_img = os.path.join(paths["img"], img_name)
        dst_img = os.path.join(paths["out"], class_names[class_id], img_name)
        

--- Step 1: Training SVM with Self-Captured Data ---
Success: svm_model.pkl generated!

--- Step 2: Evaluating Both Datasets ---

[HOG+SVM] Self Dataset Results:
              precision    recall  f1-score   support

       glass       0.03      0.03      0.03        37
       metal       0.18      0.16      0.17        37
       paper       0.08      0.05      0.06        37
     plastic       0.02      0.03      0.02        37

    accuracy                           0.07       148
   macro avg       0.08      0.07      0.07       148
weighted avg       0.08      0.07      0.07       148


[HOG+SVM] Public Dataset Results:
              precision    recall  f1-score   support

       glass       0.38      0.23      0.29       413
       metal       0.31      0.46      0.37       439
       paper       0.10      0.16      0.12       165
     plastic       0.26      0.16      0.20       446

    accuracy                           0.27      1463
   macro avg       0.26      0.26      0.2

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(r'H:\TCV_PROJECT\dataset\self capture img\resnet_train_format', transform=transform)
#train_data = datasets.ImageFolder(r'H:\TCV_PROJECT\dataset\kagel img\resnet_train_format', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


from torchvision.models import ResNet50_Weights
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 4)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()


model.train()
for epoch in range(10):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader):.4f}")

# 4. 保存！这下你就有了核心的 .pth 文件
torch.save(model.state_dict(), r'H:\TCV_PROJECT\resnet_best.pth')
print("ResNet50 训练完成，权重已保存！")

Epoch 1 Loss: 1.0234
Epoch 2 Loss: 0.2813
Epoch 3 Loss: 0.0504
Epoch 4 Loss: 0.0154
Epoch 5 Loss: 0.0088
Epoch 6 Loss: 0.0042
Epoch 7 Loss: 0.0033
Epoch 8 Loss: 0.0038
Epoch 9 Loss: 0.0147
Epoch 10 Loss: 0.0092
ResNet50 训练完成，权重已保存！


In [7]:
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in train_loader: # 先在训练集上测一下，看看模型有没有学进去
        outputs = model(inputs.to(device))
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

from sklearn.metrics import accuracy_score
print(f"ResNet 训练集准确率: {accuracy_score(y_true, y_pred):.4f}")

ResNet 训练集准确率: 1.0000


In [8]:
import cv2
import numpy as np
import joblib
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

hog = cv2.HOGDescriptor()

def extract_features(folder_path):
    features, labels = [], []
    for idx, cat in enumerate(class_names):
        p = os.path.join(folder_path, cat)
        if not os.path.exists(p): continue
        for img_n in os.listdir(p):
            img = cv2.imread(os.path.join(p, img_n))
            if img is None: continue
            img = cv2.resize(img, (128, 128))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            features.append(hog.compute(gray).flatten())
            labels.append(idx)
    return np.array(features), np.array(labels) # 确保返回 2D 数组，修复 ValueError

# 1. 训练模型
print("正在从 Self 数据集提取特征并训练 SVM...")
X_train, y_train = extract_features(datasets_config["Self"]["out"])
svm_model = LinearSVC(max_iter=2000)
svm_model.fit(X_train, y_train)

# 2. 保存权重文件
joblib.dump(svm_model, r'H:\TCV_PROJECT\svm_model.pkl')
print("SVM 权重已保存为 svm_model.pkl")

正在从 Self 数据集提取特征并训练 SVM...
SVM 权重已保存为 svm_model.pkl


In [9]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. 严格检查路径
train_dir = r'H:\TCV_PROJECT\dataset\self capture img\resnet_train_format'
val_dir = r'H:\TCV_PROJECT\dataset\self capture img\resnet_val_format'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. 加载数据
train_set = datasets.ImageFolder(train_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_set = datasets.ImageFolder(val_dir, transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=False)

# 3. 初始化并立刻训练 (使用 weights 修复之前的警告)
from torchvision.models import ResNet50_Weights
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 4)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

print("--- 开始紧急训练 (5 Epochs) ---")
model.train()
for epoch in range(5):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} 完成")

# 4. 训练完立刻在内存里进行评估 (不通过保存文件)
print("\n--- 正在生成最终报告 ---")
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs.to(device))
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=val_set.classes))

# 5. 最后顺手存一下，以防万一
torch.save(model.state_dict(), r'H:\TCV_PROJECT\resnet_best.pth')

--- 开始紧急训练 (5 Epochs) ---
Epoch 1 完成
Epoch 2 完成
Epoch 3 完成
Epoch 4 完成
Epoch 5 完成

--- 正在生成最终报告 ---
              precision    recall  f1-score   support

       glass       0.00      0.00      0.00      37.0
       metal       0.00      0.00      0.00      37.0
       paper       0.00      0.00      0.00      37.0
     plastic       0.00      0.00      0.00      37.0

    accuracy                           0.00     148.0
   macro avg       0.00      0.00      0.00     148.0
weighted avg       0.00      0.00      0.00     148.0

